# Industry Accelerators - Financial Markets Customer Attrition Prediction Model

### Introduction

Now that we have built the machine learning model, stored and deployed it using [ibm-watson-machine-learning](http://ibm-wml-api-pyclient.mybluemix.net), we can use the model to score new data. 



Before executing this notebook on IBM Cloud , you need to:<br>
1) When you import this project on an IBM Cloud environment, a project access token should be inserted at the top of this notebook as a code cell. <br>
If you do not see the cell above, Insert a project token: Click on **More -> Insert project token** in the top-right menu section and run the cell <br>

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)
2) Provide your IBM Cloud API key in the subsequent cell<br>
3) You can then step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.<br>


#### Insert IBM Cloud API key
Your Cloud API key can be generated by going to the [API Keys section of the Cloud console](https://cloud.ibm.com/iam/apikeys). From that page, scroll down to the API Keys section, and click Create an IBM Cloud API key. Give your key a name and click Create, then copy the created key and paste it below. 

If you are running this notebook on cloud pak for data on-prem, leave the ibmcloud_api_key field blank.

In [1]:
ibmcloud_api_key = ''

In [5]:
try:
    project
except NameError:
    # READING AND WRITING PROJECT ASSETS
    import project_lib
    project = project_lib.Project() 

# Create and Test Scoring Pipeline


In this notebook we will:

* Programmatically get the ID's for the deployment space and model deployment that were created in the **`2-model-training`** notebook.
* Promote assets required for scoring new data into the deployment space.
* Create a deployable function which will take raw data for scoring, prep it into the format required for the model and score it.
* Deploy the function.
* Create the required payload, invoke the deployed function and return predictions. <br>


In [12]:
import pandas as pd
import datetime
from ibm_watson_machine_learning import APIClient
import os



if ibmcloud_api_key != '':
    wml_credentials = {
        "apikey": ibmcloud_api_key,
        "url": 'https://' + os.environ['RUNTIME_ENV_REGION'] + '.ml.cloud.ibm.com'
    }
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "3.5"
     }
client = APIClient(wml_credentials)


### User Inputs

Enter the csv filename with raw data to be scored. The file will be downloaded to the local path to promote it to deployment space later.

In [13]:
filename='customer_history.csv'

In [14]:
f = open(filename, 'w+b')
f.write(project.get_file(filename).getbuffer())
f.close()

### Set up Deployment Space, Deployments and Assets

The following code programmatically gets the deployment space and the model deployment details which were created in **2-model-training**. 
We use the space name and deployment names that were used when creating the deployments as specified below. 
If multiple deployments within the selected space have the same name, the most recently created deployment is used. 

Alternatively, the user can manually enter the space and deployment id's.

The code also promotes some assets into the deployment space, specifically, the dataset with raw data for scoring, the python script file which is used for prepping the data and the metadata that was stored when prepping the data. By promoting these assets into the deployment space, they are available and can be accessed by the deployed function. 

In [15]:
space_name = 'Customer Attrition Space'
model_name = 'Customer Attrition Prediction Model'
deployment_name = 'Customer-Attrition-Prediction-Model-Deployment'

Get the space we are working in, which is found using the name that were hardcoded in **2-model-training**. 
If the user would like to use a different space manually set the **space_id**.

Set the space as the default space for working.

In [16]:
l_space_details = []
l_space_details_created_times = []
for space_details in client.spaces.get_details()['resources']:
    if space_details['entity']['name'] == space_name:
        space_id=space_details['metadata']['id']

# set this space as default space
client.set.default_space(space_id)

'SUCCESS'

Get the deployment id. If there are multiple deployments with the same name in the same space, we take the latest.

In [17]:
l_deployment_details = []
l_deployment_details_created_times = []

for deployment in client.deployments.get_details()['resources']:
        

        if deployment['entity']['name'] == deployment_name:            
                l_deployment_details.append(deployment)
                l_deployment_details_created_times.append(datetime.datetime.strptime(deployment['metadata']['created_at'],  '%Y-%m-%dT%H:%M:%S.%fZ'))
                
                

# get the index of the latest created date from the list and use that to get the deployment_id
list_latest_index = l_deployment_details_created_times.index(max(l_deployment_details_created_times))
deployment_id = l_deployment_details[list_latest_index]['metadata']['id']
print("Deployment ID of",deployment_name,"is",deployment_id)

Deployment ID of Customer-Attrition-Prediction-Model-Deployment is bdce9bcf-2cd9-40d2-b636-f399ae38476c


### Promote Assets to Deployment space

Promote the assets into the deployment space. We will use the prep script for getting the raw data into the format required for scoring. We also need the prep metadata that was saved as json during the prep for training, this ensures that the user inputs specified for prepping the data for training are the same used for scoring. We add these assets into the deployment space.  Also store the raw data dataset in the deployment space.

In [10]:
# we will use the prep script for getting the raw data into the format required for scoring
# we also need the prep metadata that was saved as json during the prep for training - this ensures that the user inputs specified for prepping the data for training are the same used for scoring
# we need to add these files into the deployment space

asset_details_json = client.data_assets.create('training_user_inputs_and_prepped_column_names.json', file_path='training_user_inputs_and_prepped_column_names.json')
asset_details_script = client.data_assets.create('attrition_prep.py', file_path='attrition_prep.py')
        
asset_details_dataset = client.data_assets.create(filename, file_path=filename)

Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS


In [19]:
client.data_assets.list()

--------------------------------------------------  ----------  --------  ------------------------------------
NAME                                                ASSET_TYPE  SIZE      ASSET_ID
training_user_inputs_and_prepped_column_names.json  data_asset  4337      637e3fab-a666-419d-8db3-c039ef057e95
attrition_prep.py                                   data_asset  35558     31739b7c-385c-4441-a9ab-2267660572c2
customer_history.csv                                data_asset  22543588  2b5fcd66-b86e-494a-a1ae-03bb0334ca31
--------------------------------------------------  ----------  --------  ------------------------------------


## Create the Deployable Function

Functions can be deployed in Watson Machine Learning in the same way models can be deployed. The python client or REST API can be used to send data to the deployed function. Using the deployed function allows us to prepare the data and pass it to the model for scoring all within the deployed function.

We start off by creating the dictionary of default parameters to be passed to the function. We get the ID's of all assets that have been promoted into the deployment space. We also add the model deployment ID and space ID information into the dictionary.

In [12]:
# get the assets that were stored in the space - in this version of the package we need to manually assign the id
metadata_id = asset_details_json['metadata']['guid']
prep_id = asset_details_script['metadata']['guid']
dataset_id = asset_details_dataset['metadata']['guid']

In [13]:
assets_dict = {'dataset_asset_id' : dataset_id, 'metadata_asset_id' : metadata_id, 
                   'prep_script_asset_id' : prep_id, 'dataset_name' : filename}

In [14]:
ai_parms = {'wml_credentials' : wml_credentials,'space_id' : space_id, 'assets' : assets_dict, 'model_deployment_id' : deployment_id}

### Scoring Pipeline Function

The function below takes new customers to be scored as a payload. It preps the customer raw data, loads the model, executes the model scoring and generates the predictions for attrition. 

The following rules are required to make a valid deployable function:

* The deployable function must include a nested function named "score".
* The score function accepts a list.
* The list must include an array with the name "values".
* The score function must return an array with the name "predictions", with a list as the value, which in turn contains an array with the name "values". Example: ```{"predictions" : [{'values' : }]}```
* We pass default parameters into the function, credentials and space detail, details of the assets that were promoted into the space and also the model deployment guid. 
* The assets are downloaded into the deployment space and imported as variables. The raw data to be scored is then prepared and the function calls the model deployment endpoint to score and return predictions. 

In [15]:
def scoring_pipeline(parms=ai_parms):
    
    import pandas as pd
    import requests
    import os
    import json
    
    from ibm_watson_machine_learning import APIClient
    client = APIClient(parms["wml_credentials"])
    client.set.default_space(parms['space_id'])
     
    
    # call the function to download the stored dataset asset and return the path
    dataset_path = client.data_assets.download(parms['assets']['dataset_asset_id'], parms['assets']['dataset_name'])
    df_raw = pd.read_csv(dataset_path, infer_datetime_format=True, 
                             parse_dates=['CUSTOMER_RELATIONSHIP_START_DATE', 
                                              'CUSTOMER_SUMMARY_END_DATE','CUSTOMER_SUMMARY_START_DATE'])

    # call the function to download the prep script and return the path
    prep_script_path = client.data_assets.download(parms['assets']['prep_script_asset_id'], 'prep_data_script.py')
    # remove the rest of path and .py at end of file name to get the name of the script for importing
    script_name = os.path.basename(prep_script_path).replace('.py', '')
    
    # call the function to download the prep metadata and return the path
    metadata_path = client.data_assets.download(parms['assets']['metadata_asset_id'], 'user_inputs.json')
    
    def prep(cust_id, sc_end_date):
        
        import requests
        import os
        # import the prep script that we downloaded into the deployment space
        prep_data_script = __import__(script_name)
        
        with open(metadata_path, 'r') as f:
            user_inputs_dict = json.load(f)
        
        globals().update(user_inputs_dict)
                  
        input_df = df_raw[df_raw[granularity_key] == cust_id]
        
        scoring_prep = prep_data_script.AttritionPrep('score', effective_date=sc_end_date, feature_attributes=feature_attributes,
                             derive_column_list=derive_column_list,
                             granularity_key=granularity_key, target_attribute=target_attribute,
                             status_attribute=status_attribute,
                             funds_attribute=funds_attribute, date_customer_joined=date_customer_joined,
                             customer_end_date=customer_end_date, customer_start_date=customer_start_date,
                             period_attribute=period_attribute, status_flag_attrition=status_flag_attrition,
                             AUM_reduction_threshold=AUM_reduction_threshold,
                             forecast_horizon=forecast_horizon, observation_window=observation_window,
                             sum_list=sum_list, cat_threshold=cat_threshold)
        
        prepped_data = scoring_prep.prep_data(input_df, 'score')
        
        if prepped_data is None:
            print("Data prep filtered out customer data. Unable to score.", file=sys.stderr)
            return None
    
        # handle empty data
        if prepped_data.shape[0] == 0:
            print("Data prep filtered out customer data. Unable to score.", file=sys.stderr)
            return None

        to_drop_corr = ['CUSTOMER_SUMMARY_FUNDS_UNDER_MANAGEMENT_mean', 'CUSTOMER_SUMMARY_FUNDS_UNDER_MANAGEMENT_min',
                            'CUSTOMER_SUMMARY_FUNDS_UNDER_MANAGEMENT_max', 'CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_min',
                            'CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_max', 'CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_sum',
                            'CUSTOMER_ANNUAL_INCOME', 'CUSTOMER_NUMBER_OF_DEPENDENT_CHILDREN',
                            'NUM_ACCOUNTS_WITH_RISK_TOLERANCE_MODERATE', 'NUM_ACCOUNTS_WITH_RISK_TOLERANCE_HIGH',
                            'NUM_ACCOUNTS_WITH_RISK_TOLERANCE_VERY_LOW', 'NUM_ACCOUNTS_WITH_RISK_TOLERANCE_LOW', 
                            'CUSTOMER_TENURE', 'NUM_ACCOUNTS_WITH_INVESTMENT_OBJECTIVE_PLANNING',
                            'NUM_ACCOUNTS_WITH_INVESTMENT_OBJECTIVE_SECURITY','CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_max_min_ratio',
                            'CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_DEPOSITS_current_vs_6_months_ago']
        
        # don't need to include target variable for scoring
        cols_used_for_training.remove(target_attribute)

        # if a column does not exist in scoring but is in training, add the column to scoring dataset
        for col in cols_used_for_training:
            if col not in list(prepped_data.columns):
                prepped_data[col] = 0

        # if a column exists in scoring but not in training, delete it from scoring dataset
        for col in list(prepped_data.columns):
            if col not in cols_used_for_training:
                prepped_data.drop(col, axis=1, inplace=True)

        # make sure order of scoring columns is same as training dataset
        prepped_data = prepped_data[cols_used_for_training]
        
        prepped_data = prepped_data.drop(to_drop_corr, axis=1)
        
        return prepped_data
        
    def score(payload):
        
        import json
        try:
            sc_end_date = payload['input_data'][0]['values'][0][1]
        except:
            sc_end_date="2018-09-30"
            
        cust_id = payload['input_data'][0]['values'][0][0]
        
        prepped_data = prep(cust_id, sc_end_date)
        
        if prepped_data is None:
            return {"predictions" : [{'values' : 'Data prep filtered out customer data. Unable to score.'}]}
        else:

            scoring_payload = {"input_data":  [{ "values" : prepped_data.values.tolist()}]}
            
            response_scoring = client.deployments.score(parms['model_deployment_id'], scoring_payload)
            result=  response_scoring
            print(result['predictions'][0]['values'][0][1][1])
            result["Probability_of_Attrition"]=str(round(result['predictions'][0]['values'][0][1][1]*100,2))+"%"
            return {"predictions" : [{'values' : result}]}

    return score

### Deploy the Function

The user can specify the name of the function and deployment in the code below. 

In [16]:
# store the function and deploy it 
function_name = 'Customer Attrition Prediction Scoring Function'
function_deployment_name = 'Customer-Attrition-Prediction-Scoring-Function-Deployment'

We use tags, input data schemas, output data schemas and software specifications in the metadata to store the function. Input data schemas provides an easy option to input data to score in the deployment space. Example to create a metatadata to store the function can be viewed using `client.repository.FunctionMetaNames.get_example_values()`, similarly example to create a metatadata to deploy the function can be viewed using `client.deployments.ConfigurationMetaNames.get_example_values()` <br>
The Software Specification refers to the runtime used in the Notebook, WML training and WML deployment. We use the software specification `default_py3.7` to store the function. We get the ID of the software specification and include it in the metadata when storing the function. Available Software specifications can be retrieved using `client.software_specifications.list()`.


In [17]:

software_spec_id = client.software_specifications.get_id_by_name("default_py3.7")

In [18]:
# add the metadata for the function and deployment    
meta_data = {
    client.repository.FunctionMetaNames.NAME : function_name,
    client.repository.FunctionMetaNames.TAGS : ['attrition_scoring_pipeline_function_tag'],
    client.repository.FunctionMetaNames.INPUT_DATA_SCHEMAS:[{'id': '1','type': 'struct','fields': [{'name': 'CUSTOMER ID', 'type': 'int'},{'name': 'sc_end_date', 'type': 'date'}]}],
    client.repository.FunctionMetaNames.OUTPUT_DATA_SCHEMAS: [{'id': '1','type': 'struct','fields': [{'name': 'Probability_of_Attrition','type': 'double'}]}],
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: software_spec_id

}

function_details = client.repository.store_function(meta_props=meta_data, function=scoring_pipeline)

function_id = function_details["metadata"]["id"]

meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: function_deployment_name,
    client.deployments.ConfigurationMetaNames.TAGS : ['attrition_scoring_pipeline_function_deployment_tag'],
    client.deployments.ConfigurationMetaNames.DESCRIPTION:"Customer Attrition Scoring Function which will take raw data for scoring, prep it into the format required for the model and score it to return attrition probability of the customer.",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the function
function_deployment_details = client.deployments.create(artifact_uid=function_id, meta_props=meta_props)




#######################################################################################

Synchronous deployment creation for uid: 'a0911417-1580-4106-89df-deacf837fb77' started

#######################################################################################


initializing.....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='baa30d56-3e3e-48a7-8c59-bc07b6bdfd9d'
------------------------------------------------------------------------------------------------




### Score New Data

Get the guid of the deployed function, create the payload and use the python client to score the data. The deployed function returns the classification prediction along with the probabilities. 

The payload contains two values. The first is the effective date for scoring. This is the date that the prediction is computed. The scoring observation window and forecast horizon are calculated from this date. The second value contains the ID of the customer who we would like to make the prediction for. 

In [19]:
scoring_deployment_id = client.deployments.get_uid(function_deployment_details)
client.deployments.get_details(scoring_deployment_id)

{'entity': {'asset': {'id': 'a0911417-1580-4106-89df-deacf837fb77'},
  'custom': {},
  'deployed_asset_type': 'function',
  'description': 'Customer Attrition Scoring Function which will take raw data for scoring, prep it into the format required for the model and score it to return attrition probability of the customer.',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'XS', 'num_nodes': 1},
  'name': 'Customer-Attrition-Prediction-Scoring-Function-Deployment',
  'online': {},
  'space_id': '688ecf7d-253a-4a49-a927-7d9913c37817',
  'status': {'online_url': {'url': 'https://eu-de.ml.cloud.ibm.com/ml/v4/deployments/baa30d56-3e3e-48a7-8c59-bc07b6bdfd9d/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-03-23T14:32:26.046Z',
  'description': 'Customer Attrition Scoring Function which will take raw data for scoring, prep it into the format required for the model and score it to return attrition probability of the customer.',
  'id': 'baa30d56-3e3e-48a7-8c59-bc07b6b

In [20]:
cust_id = 1008

payload = [{"fields":["CUSTOMER_ID","sc_end_date"],'values' : [[cust_id,"2018-09-30"]]}]

payload_metadata = {client.deployments.ScoringMetaNames.INPUT_DATA: payload}
# score
funct_output = client.deployments.score(scoring_deployment_id, payload_metadata)
funct_output

{'predictions': [{'values': {'predictions': [{'fields': ['prediction',
       'probability'],
      'values': [[0, [0.6941542677380026, 0.3058457322619974]]]}],
    'Probability_of_Attrition': '30.58%'}}]}

**The R Shiny Dashboard invokes this scoring pipeline for visualizing the results.**<br>
**Follow the instructions from Readme to launch R-Shiny dashboard.**


<hr>

Sample Materials, provided under <a href="https://github.com/IBM/Industry-Accelerators/blob/master/CPD%20SaaS/LICENSE" target="_blank" rel="noopener noreferrer">license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2019, 2021. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>